# <div align="center" style="color: #ff5733;">CredoLab Data</div>

**Requestor** -- `Biswa`<br><br>
**Purpose** -- `Share the data with Two ukranian data scientist` <br><br>
**Description** -- `Pull the Credolab data for the users that onboarded between 1st July 2023 till 30th June 2024. No feature engineering needed, just pull the data at user level, and attach the flag_bad (FSPD30 for both Quick and SIL). You don't even need to get the App category wise counts. I will share it with Vadym and Oleh so that they can start developing the Apps model` <br><br>

**Created By:   -** Dwaipayan Chakroborti <br><br>
**Date  :** 26-09-2024

# Define Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """   
WITH
  b AS (
  SELECT
    loanAccountNumber,
    min_inst_def30,
    obs_min_inst_def30
  FROM
    prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data
  WHERE
    obs_min_inst_def30 >= 2),
lmt as
(SELECT
  lmt.loanAccountNumber,
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.tsa_onboarding_time,
  lmt.startApplyDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.isTermsAndConditionsAccepted,
  lmt.disbursementDateTime,
  lmt.flagDisbursement,
  lmt.loanPaidStatus,
  case when b.obs_min_inst_def30 >=2 and b.min_inst_def30 in (1,2) then lmt.loanAccountNumber end FSPD30_loancnt,
  case when b.obs_min_inst_def30 >=2 then lmt.loanAccountNumber end obsFSPD30_loancnt
FROM
  `risk_credit_mis.loan_master_table` lmt
INNER JOIN
  b
ON
  lmt.loanAccountNumber = b.loanAccountNumber 
)
select 
distinct
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.tsa_onboarding_time,
  lmt.startApplyDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.isTermsAndConditionsAccepted,
  lmt.disbursementDateTime,
  lmt.flagDisbursement,
  lmt.loanPaidStatus,
  t3.creditScoreUpdated   ,
  t3.fraudScore   ,	
  t3.fraudScoreUpdated    ,
  t3.calculateddate   ,
  t4.run_date ,
  ca.package_name ,
  ca.first_install_time    ,
  ca.last_update_time      ,
  t4.GeneralInfo.brand     ,
  t4.Hardware.device__brand   ,
  t4.Hardware.device__manufacturer   ,
  t4.Hardware.device__model,
  t4.GeneralData.telephony_info__network_operator_name,
  t4.GeneralData.telephony_info__network_operator,
  t4.GeneralData.sim_operator_name,
  ptat.Category,
  -- ptat.Rating,
  case when ptat.Rating = 'rated for 3+' then 1 else 0 end rated_for_3_plus,
  case when ptat.Rating = 'rated for 7+' then 1 else 0 end rated_for_7_plus,
  case when ptat.Rating = 'rated for 12+' then 1 else 0 end rated_for_12_plus,
  case when ptat.Rating = 'rated for 16+' then 1 else 0 end rated_for_16_plus,
  case when ptat.Rating = 'rated for 18+' then 1 else 0 end rated_for_18_plus,
  case when ptat.Rating = 'undefined' then 1 else 0 end undefined,
  case when ptat.Rating = 'unrated' then 1 else 0 end unrated,
  case when ptat.Rating is null then 1 else 0 end Rating_Not_Available,
  ptat.Is_Paid,
  lmt.FSPD30_loancnt,     ---- FSPD30 = 1 when this value is not null(provided this as there were be duplicate rows in this dataset because of package name)
  lmt.obsFSPD30_loancnt   ---- obsFSPD30 = 1 when this value is not null (provided this as there were be duplicate rows in this dataset because of package name)
from lmt
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON lmt.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
inner join
`prj-prod-dataplatform.core_raw.loan_accounts` loan
on loan.CUSTOMERID = lmt.customerId
 INNER JOIN
(select deviceId, af.package_name as package_name, af.first_install_time as first_install_time , af.last_update_time as last_update_time from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
unnest(Application) as af) ca
ON ca.deviceId = t3.refno
LEFT JOIN prj-prod-dataplatform.dap_ds_poweruser_playground.PH_Tonikbank_Application_Temp ptat
ON REGEXP_REPLACE(ca.package_name, r'[ ._]', '') = REGEXP_REPLACE(ptat.Package_Name, r'[ ._]', '')
where date(lmt.startApplyDateTime) >='2024-06-01'   ---- Please change the date as per your requirement. This is Loan Application Apply Date
and lmt.FSPD30_loancnt is not null
order by lmt.customerId
limit 1000   --- Please remove this when running the query
;

"""

In [3]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 35875c7d-c9fb-4fbd-a273-2a6e5c4d99c8 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [4]:
df.head()

,customerId,digitalLoanAccountId,loanAccountNumber,tsa_onboarding_time,startApplyDateTime,termsAndConditionsSubmitDateTime,isTermsAndConditionsAccepted,disbursementDateTime,flagDisbursement,loanPaidStatus,...,rated_for_7_plus,rated_for_12_plus,rated_for_16_plus,rated_for_18_plus,undefined,unrated,Rating_Not_Available,Is_Paid,FSPD30_loancnt,obsFSPD30_loancnt
0,1302142,d3038dd0-0e81-4983-91d8-9fc9065a8ed1,60813021420026,2022-04-04 21:16:19,2024-06-01 08:13:50,2024-06-05 09:08:57,1,2024-06-05 09:43:16,1,Normal,...,0,0,0,0,0,0,1,None,60813021420026,60813021420026
1,1302142,d3038dd0-0e81-4983-91d8-9fc9065a8ed1,60813021420026,2022-04-04 21:16:19,2024-06-01 08:13:50,2024-06-05 09:08:57,1,2024-06-05 09:43:16,1,Normal,...,0,0,0,0,0,0,1,None,60813021420026,60813021420026
2,1302142,d3038dd0-0e81-4983-91d8-9fc9065a8ed1,60813021420026,2022-04-04 21:16:19,2024-06-01 08:13:50,2024-06-05 09:08:57,1,2024-06-05 09:43:16,1,Normal,...,0,0,0,0,0,0,1,None,60813021420026,60813021420026
3,1302142,d3038dd0-0e81-4983-91d8-9fc9065a8ed1,60813021420026,2022-04-04 21:16:19,2024-06-01 08:13:50,2024-06-05 09:08:57,1,2024-06-05 09:43:16,1,Normal,...,0,0,0,0,0,0,0,free,60813021420026,60813021420026
4,1302142,d3038dd0-0e81-4983-91d8-9fc9065a8ed1,60813021420026,2022-04-04 21:16:19,2024-06-01 08:13:50,2024-06-05 09:08:57,1,2024-06-05 09:43:16,1,Normal,...,0,0,0,0,0,0,1,None,60813021420026,60813021420026


In [5]:
df.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\CredolabData\Data\credolabsampledata20240927.csv", index = False)